In [1]:
from bokeh.io import output_file, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, LinearColorMapper
from bokeh.models.widgets import DataTable, TableColumn
from bokeh.palettes import Greens5, Purples5
from bokeh.layouts import Row
from bokeh.embed import components
import numpy as np

In [2]:
output_notebook()
#output_file('BeenThereDoneThat.html', title="Andres' Map")

Loading BokehJS ...

In [3]:
import geopandas as gp

In [4]:
region = gp.read_file('data/ne_50m_admin_0_countries.shp')

In [5]:
region.head()

,abbrev,abbrev_len,adm0_a3,adm0_a3_is,adm0_a3_un,adm0_a3_us,adm0_a3_wb,adm0_dif,admin,brk_a3,...,su_dif,subregion,subunit,tiny,type,un_a3,wb_a2,wb_a3,wikipedia,woe_id
0,Aruba,5.0,ABW,ABW,-99.0,ABW,-99.0,1.0,Aruba,ABW,...,0.0,Caribbean,Aruba,4.0,Country,533,AW,ABW,-99.0,-99.0
1,Afg.,4.0,AFG,AFG,-99.0,AFG,-99.0,0.0,Afghanistan,AFG,...,0.0,Southern Asia,Afghanistan,-99.0,Sovereign country,004,AF,AFG,-99.0,-99.0
2,Ang.,4.0,AGO,AGO,-99.0,AGO,-99.0,0.0,Angola,AGO,...,0.0,Middle Africa,Angola,-99.0,Sovereign country,024,AO,AGO,-99.0,-99.0
3,Ang.,4.0,AIA,AIA,-99.0,AIA,-99.0,1.0,Anguilla,AIA,...,0.0,Caribbean,Anguilla,-99.0,Dependency,660,-99,-99,-99.0,-99.0
4,Alb.,4.0,ALB,ALB,-99.0,ALB,-99.0,0.0,Albania,ALB,...,0.0,Southern Europe,Albania,-99.0,Sovereign country,008,AL,ALB,-99.0,-99.0


In [6]:
visited_countries = ['Cambodia', 'China', 'Hong Kong S.A.R.','India','Indonesia','Japan', 'South Korea', 'Laos', 'Macao S.A.R', 'Malaysia','Mongolia', 'Singapore','Taiwan', 'Thailand', 'Vietnam','Morocco', 'El Salvador','Guatemala','Denmark','France','Germany','Iceland', 'Ireland', 'Italy','Norway', 'Russia',
'Switzerland','United Kingdom','Vatican', 'Turkey', 'Mexico', 'United States of America']

In [7]:
len(visited_countries)

32

In [8]:
len(region[region['admin'].isin(visited_countries)])

32

In [9]:
region[region['admin'].str.contains('Vat')]

,abbrev,abbrev_len,adm0_a3,adm0_a3_is,adm0_a3_un,adm0_a3_us,adm0_a3_wb,adm0_dif,admin,brk_a3,...,su_dif,subregion,subunit,tiny,type,un_a3,wb_a2,wb_a3,wikipedia,woe_id
228,Vat.,4.0,VAT,VAT,-99.0,VAT,-99.0,0.0,Vatican,VAT,...,0.0,Southern Europe,Vatican,4.0,Sovereign country,336,-99,-99,0.0,-99.0


In [10]:
region['Visited'] = False

In [11]:
# This returns an actual view, rather than a copy to which you cannot assign a value. LOC[rows, cols]
region.loc[region['admin'].isin(visited_countries),'Visited'] = True

In [12]:
# region[region['Visited'] == True]

In [13]:
def getXYCoords(geometry, coord_type):
    """Returns the x or y coordinates of a provided geometry"""
    if coord_type =='x':
        return geometry.coords.xy[0]
    elif coord_type =='y':
        return geometry.coords.xy[1]

In [14]:
def getPolyCoords(geometry, coord_type):
    """Returns the exterior of a polygon"""
    ext = geometry.exterior
    return getXYCoords(ext, coord_type)

In [15]:
def getLineCoords(geometry, coord_type):
    """Returns the coordinates for the line"""
    return getXYCoords(geometry, coord_type)

In [16]:
def getPointCoords(geometry, coord_type):
    """Returns the coordinates for a point"""
    if coord_type == 'x':
        return geometry.x
    elif coord_type == 'y':
        return geometry.y

In [17]:
def multiGeomHandler(multi_geo, coord_type):
    """Handles Multi-points, Multi-lines, Multi-Polygons.
    Original reference is here:
    https://automating-gis-processes.github.io/Lesson5-interactive-map-Bokeh-advanced-plotting.html
    It has been refactored to not need the extra third argument of geom_type.
    """
    g_type = multi_geo.geom_type
    for i, part in enumerate(multi_geo):
        # On the first part of the Multi-geometry initialize the coord_array (np.array)
        if i == 0:
            if g_type == "MultiPoint":
                coord_arrays = np.append(getPointCoords(part, coord_type), np.nan)
            elif g_type == "MultiLineString":
                coord_arrays = np.append(getLineCoords(part, coord_type), np.nan)
            elif g_type == "MultiPolygon":
                coord_arrays = np.append(getPolyCoords(part, coord_type), np.nan)
        else:
            if g_type == "MultiPoint":
                coord_arrays = np.concatenate([coord_arrays, np.append(getPointCoords(part, coord_type), np.nan)])
            elif g_type == "MultiLineString":
                coord_arrays = np.concatenate([coord_arrays, np.append(getLineCoords(part, coord_type), np.nan)])
            elif g_type == "MultiPolygon":
                coord_arrays = np.concatenate([coord_arrays, np.append(getPolyCoords(part, coord_type), np.nan)])
    
    # Return the coordinates 
    return coord_arrays

In [18]:
def getCoords(row, geo_col, coord_type):
    """
    Returns coordinates ('x' or 'y') of a geometry (Point, LineString or Polygon) as a list (if geometry is LineString or Polygon). 
    Can handle also MultiGeometries.
    """
    # Get geometry
    geom = row[geo_col]
    
    # Check the geometry type
    gtype = geom.geom_type
    
    # "Normal" geometries
    # -------------------
    
    if gtype == "Point":
        return getPointCoords(geom, coord_type)
    elif gtype == "LineString":
        return list( getLineCoords(geom, coord_type) )
    elif gtype == "Polygon":
        return list( getPolyCoords(geom, coord_type) )
        
    # Multi geometries
    # ----------------
    
    else:
        return list( multiGeomHandler(geom, coord_type) ) 

In [19]:
region['x'] = region.apply(getCoords, geo_col='geometry', coord_type= 'x', axis = 1)
region['y'] = region.apply(getCoords, geo_col='geometry', coord_type= 'y', axis = 1)

In [20]:
# Converting from lists to np.arrays that bokeh utilizes
region['x'] = region['x'].apply(np.asarray)
region['y'] = region['y'].apply(np.asarray)

In [21]:
region_source = ColumnDataSource(region.loc[region['Visited'] == False, ('x','y','admin','un_a3')])
visited_source = ColumnDataSource(region.loc[region['Visited'] == True, ('x','y','admin','un_a3')])

In [22]:
hover = HoverTool(tooltips=[("Country", "@admin")])

In [23]:
tools = [hover, 'pan','wheel_zoom','reset']

In [24]:
visited_colors = LinearColorMapper(Purples5[:3])
not_visited_colors = LinearColorMapper(Greens5[:3])

In [25]:
p = figure(title="Countries visited", plot_width= 960, plot_height = 600, tools=tools, output_backend="webgl", logo=None)

In [26]:
not_visited = p.patches(xs='x', ys='y', source = region_source, fill_color= {'field': 'un_a3', 'transform': not_visited_colors}, muted_alpha = 0.2, line_alpha = 0.4, legend = 'Will visit soon')
visited = p.patches(xs='x', ys='y', source = visited_source, fill_color={'field': 'un_a3', 'transform':visited_colors}, muted_alpha = 0.2, line_alpha = 0.4, legend = 'Been there, done that')
p.legend.location='center_left'
p.legend.click_policy='mute'
p.toolbar_location='above'
p.title.text_font_size = '3rem'
p.title.text_font= "Permanent Marker"
p.title.text_color="black"

In [27]:
data_table = DataTable(source= visited_source, columns = [TableColumn(field='admin', title='Countries Visited So Far')], height = 550, width = 200, row_headers=False)

In [28]:
show(Row(p, data_table))

In [29]:
# Use this to embed.
#script, div = components(Row(p, data_table))